# stream

In [ ]:
open rust.rust_operators

In [ ]:
//// test

open testing

## stream

### stream

In [ ]:
union rec stream t =
    | StreamCons : t * (() -> stream t)
    | StreamNil

### fold

In [ ]:
inl fold fn init s =
    inl rec body acc = function
        | StreamCons (st, fn') => loop (fn acc st) (fn' ())
        | StreamNil => acc
    and inl loop acc = join_body body acc
    loop init s

### fold_back

In [ ]:
inl fold_back fn s init =
    inl rec body acc = function
        | StreamCons (st, fn') => fn st (loop acc (fn' ()))
        | StreamNil => acc
    and inl loop acc = join_body body acc
    loop init s

### to_list

In [ ]:
inl to_list s =
    (s, [])
    ||> fold_back fun x acc =>
        x :: acc

### rev

In [ ]:
inl rev s =
    (StreamNil, s)
    ||> fold fun s x =>
        StreamCons (x, fun () => s)

### from_list

In [ ]:
inl from_list list =
    (list, StreamNil)
    ||> listm.foldBack fun x acc =>
        StreamCons (x, fun () => acc)

In [ ]:
//// test

listm.init 3i32 id
|> from_list
|> rev
|> to_list
|> _assert_eq [ 2; 1; 0 ]

__assert_eq / actual: UH0_1 (2, UH0_1 (1, UH0_1 (0, UH0_0))) / expected: UH0_1 (2, UH0_1 (1, UH0_1 (0, UH0_0)))


### try_item

In [ ]:
inl try_item i s =
    inl rec body i = function
        | StreamCons (x, _) when i <= 0 => Some x
        | StreamCons (_, fn) => loop (i - 1) (fn ())
        | StreamNil => None
    and inl loop acc s' =
        match var_is acc, var_is s' with
        | false, false => body acc s'
        | _ =>
            inl acc = dyn acc
            inl s' = dyn s'
            join body acc s'
    loop i s

inl item i =
    try_item i >> optionm.value

In [ ]:
//// test

listm.init 10i32 id
|> from_list
|> item 9i32
|> _assert_eq 9

__assert_eq / actual: 9 / expected: 9


### new_infinite_stream

In [ ]:
inl new_infinite_stream fn =
    inl rec loop n =
        StreamCons (fn n, fun () => loop (n + 1))
    loop 0

inl new_infinite_stream_ fn =
    let rec loop n =
        StreamCons (fn n, fun () => loop (n + 1))
    loop 0

In [ ]:
//// test

new_infinite_stream print_and_return
|> item 4i32
|> _assert_eq 4i32

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
print_and_return / x: 4
__assert_eq / actual: 4 / expected: 4


### new_finite_stream

In [ ]:
inl new_finite_stream fn max =
    inl rec loop n =
        if n >= max
        then StreamNil
        else StreamCons (fn n, fun () => loop (n + 1))
    loop 0

### memoize

In [ ]:
union memoized_stream t =
    | NotComputed : () -> stream t
    | Computed : stream t

inl memoize s =
    inl rec body s =
        inl state = mut (NotComputed s)
        fun () =>
            match *state with
            | Computed x => x
            | NotComputed fn =>
                inl new_state =
                    match fn () with
                    | StreamNil => StreamNil
                    | StreamCons (x, fn) => StreamCons (x, loop fn)
                state <- Computed new_state
                new_state
    and inl loop s' = join_body_unit body s s'
    loop (fun () => s)

In [ ]:
//// test

inl memo_stream = new_finite_stream print_and_return 10 |> memoize

memo_stream ()
|> item 3i32
|> _assert_eq 3i32

memo_stream ()
|> item 5i32
|> _assert_eq 5i32

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
__assert_eq / actual: 3 / expected: 3
print_and_return / x: 4
print_and_return / x: 5
__assert_eq / actual: 5 / expected: 5


In [ ]:
//// test

inl memo_stream = new_infinite_stream_ print_and_return |> memoize

memo_stream ()
|> item 3i32
|> _assert_eq 3i32

memo_stream ()
|> item 5i32
|> _assert_eq 5i32

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
__assert_eq / actual: 3 / expected: 3
print_and_return / x: 4
print_and_return / x: 5
__assert_eq / actual: 5 / expected: 5


### unfold

In [ ]:
inl unfold f x0 =
    inl rec body x =
        match f x with
        | Some (x', y) => StreamCons (x', fun () => loop y)
        | None => StreamNil
    and inl loop x = join_body_unit body x0 x
    loop x0

### iterate

In [ ]:
inl iterate f =
    fun x => Some (x, f x)
    |> unfold

In [ ]:
//// test

iterate ((*) 2) 1i32
|> item 10i32
|> _assert_eq 1024

__assert_eq / actual: 1024 / expected: 1024


### iterate'

In [ ]:
inl iterate_map f m =
    fun x =>
        m x
        |> optionm.map fun x =>
            x, f x
    |> unfold

In [ ]:
//// test

iterate_map ((*) 2) Some 1i32
|> item 10i32
|> _assert_eq 1024

__assert_eq / actual: 1024 / expected: 1024


### take_while

In [ ]:
inl take_while cond s =
    inl rec body i = function
        | StreamCons (st, fn) when cond st i => StreamCons (st, fun () => loop (i + 1) (fn ()))
        | _ => StreamNil
    and inl loop i = join_body body i
    loop 0 s

### sum

In [ ]:
inl sum seq =
    seq |> fold (+) 0

In [ ]:
//// test

listm.init 10i32 id
|> from_list
|> sum
|> _assert_eq 45

__assert_eq / actual: 45 / expected: 45


In [ ]:
//// test

new_finite_stream print_and_return 10i32
|> take_while (fun n (_ : i32) => n < 5)
|> sum
|> _assert_eq 10

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
print_and_return / x: 4
print_and_return / x: 5
__assert_eq / actual: 10 / expected: 10


In [ ]:
//// test

new_infinite_stream print_and_return
|> take_while (fun n (_ : i32) => n < 5i32)
|> sum
|> _assert_eq 10

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
print_and_return / x: 4
print_and_return / x: 5
__assert_eq / actual: 10 / expected: 10


In [ ]:
//// test

iterate ((*) 6) 1i32
|> take_while (fun _ i => i <= 7i32)
|> to_list
|> _assert_eq [ 1i32; 6; 36; 216; 1296; 7776; 46656; 279936 ]

__assert_eq / actual: UH0_1
  (1,
   UH0_1
     (6,
      UH0_1
        (36,
         UH0_1
           (216,
            UH0_1 (1296, UH0_1 (7776, UH0_1 (46656, UH0_1 (279936, UH0_0)))))))) / expected: UH0_1
  (1,
   UH0_1
     (6,
      UH0_1
        (36,
         UH0_1
           (216,
            UH0_1 (1296, UH0_1 (7776, UH0_1 (46656, UH0_1 (279936, UH0_0))))))))


### indexed

In [ ]:
inl indexed s =
    ((StreamNil, 0), s)
    ||> fold fun (acc, i) x =>
        StreamCons ((i, x), fun () => acc), i + 1
    |> fst
    |> rev

In [ ]:
//// test

listm.init 10i32 ((*) 2)
|> from_list
|> indexed
|> item 5i32
|> _assert_eq (5i32, 10i32)

__assert_eq / actual: struct (5, 10) / expected: struct (5, 10)


### map

In [ ]:
inl map fn s =
    (s, StreamNil)
    ||> fold_back fun x acc =>
        StreamCons (fn x, fun () => acc)

In [ ]:
//// test

listm.init 10i32 id
|> from_list
|> map ((*) 2)
|> item 5i32
|> _assert_eq 10i32

__assert_eq / actual: 10 / expected: 10


### zip_with

In [ ]:
inl zip_with fn s1 s2 =
    inl rec loop s1 s2 =
        match s1, s2 with
        | StreamCons (st1, fn1), StreamCons (st2, fn2) =>
            StreamCons (fn st1 st2, fun () => loop (fn1 ()) (fn2 ()))
        | StreamNil, _ | _, StreamNil => StreamNil
    loop s1 s2

inl zip_with_ fn s1 s2 =
    let rec loop s1 s2 =
        match s1, s2 with
        | StreamCons (st1, fn1), StreamCons (st2, fn2) =>
            StreamCons (fn st1 st2, fun () => loop (fn1 ()) (fn2 ()))
        | StreamNil, _ | _, StreamNil => StreamNil
    loop s1 s2

In [ ]:
//// test

((listm.init 10i32 id |> from_list), (listm.init 10i32 ((*) 2) |> from_list))
||> zip_with (+)
|> item 2i32
|> _assert_eq 6

__assert_eq / actual: 6 / expected: 6


### zip

In [ ]:
inl zip s1 s2 =
    zip_with pair s1 s2

inl zip_ s1 s2 =
    zip_with_ pair s1 s2

In [ ]:
//// test

((listm.init 10i32 id |> from_list), (listm.init 10i32 ((*) 2) |> from_list))
||> zip
|> item 5i32
|> _assert_eq (5, 10)

__assert_eq / actual: struct (5, 10) / expected: struct (5, 10)


### unzip

In [ ]:
inl unzip s =
    inl rec body s =
        match s with
        | StreamCons ((x, y), fn) =>
            inl xs, ys = loop (fn ())
            StreamCons (x, fun () => xs), StreamCons (y, fun () => ys)
        | StreamNil => pair StreamNil StreamNil
    and inl loop x =
        if var_is x |> not
        then body x
        else
            inl x = dyn x
            join body x
    loop s

In [ ]:
//// test

listm.init 10i32 id
|> listm.map (fun x => x, x)
|> from_list
|> unzip
|> fun x, y =>
    x |> sum
    |> _assert_eq 45

    y |> sum
    |> _assert_eq 45

__assert_eq / actual: 45 / expected: 45
__assert_eq / actual: 45 / expected: 45


## rust

### io_error

In [ ]:
nominal io_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::Error\")>]\ntype std_io_Error = class end\n#else\ntype std_io_Error = string\n#endif\n"
        $'' : $'std_io_Error'
    )

### new_io_error

In [ ]:
inl new_io_error (text : string) : io_error =
    run_target_args (fun () => text) function
        | Rust _ => fun text =>
            !\\(text, $'"std::io::Error::new(std::io::ErrorKind::Other, &*$0)"')
        | _ => fun text => text |> unbox

### buf_reader

In [ ]:
nominal buf_reader t =
    `(
        backend_switch `(()) `({}) {
            Fsharp =
                (fun () =>
                    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::BufReader<$0>\")>]\n#endif\ntype std_io_BufReader<'T> = class end"
                ) : () -> ()
        }
        $'' : $'std_io_BufReader<`t>'
    )

### cursor

In [ ]:
nominal cursor t =
    `(
        backend_switch `(()) `({}) {
            Fsharp =
                (fun () =>
                    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::Cursor<$0>\")>]\n#endif\ntype std_io_Cursor<'T> = class end"
                ) : () -> ()
        }
        $'' : $'std_io_Cursor<`t>'
    )

### buf_reader_tokio

In [ ]:
nominal buf_reader_tokio t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"tokio::io::BufReader<$0>\")>]\n#endif\ntype tokio_io_BufReader<'T> = class end"
        $'' : $'tokio_io_BufReader<`t>'
    )

### new_buf_reader

In [ ]:
inl new_buf_reader forall t. (x : t) : buf_reader t =
    !\\(x, $'"std::io::BufReader::new($0)"')

### new_cursor

In [ ]:
inl new_cursor forall t. (x : t) : cursor t =
    !\($'"std::io::Cursor::new(!x)"')

### lines

In [ ]:
nominal lines t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::Lines<$0>\")>]\n#endif\ntype std_io_Lines<'T> = class end"
        $'' : $'std_io_Lines<`t>'
    )

### buf_read_lines

In [ ]:
inl buf_read_lines forall t. (buf_reader : buf_reader t) : lines (buf_reader t) =
    !\($'"std::io::BufRead::lines(!buf_reader)"')

### decode_reader_bytes

In [ ]:
nominal decode_reader_bytes t u =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"encoding_rs_io::DecodeReaderBytes<$0, $1>\")>]\n#endif\ntype encoding_rs_io_DecodeReaderBytes<'T, 'U> = class end"
        $'' : $'encoding_rs_io_DecodeReaderBytes<`t, `u>'
    )

### decode_reader_bytes_build

In [ ]:
inl decode_reader_bytes_build forall t. (x : t) : decode_reader_bytes t (am'.vec u8) =
    // !\($'"encoding_rs_io::DecodeReaderBytesBuilder::new().encoding(Some(encoding_rs::UTF_8)).build(!x)"')
    // !\($'"encoding_rs_io::DecodeReaderBytesBuilder::new().encoding(Some(encoding_rs::UTF_8)).utf8_passthru(true).build(!x)"')
    !\\(x, $'"encoding_rs_io::DecodeReaderBytesBuilder::new().utf8_passthru(true).build($0)"')
    // !\($'"encoding_rs_io::DecodeReaderBytes::new(!x)"')

### buf_reader_read

In [ ]:
inl buf_reader_read forall el dim. (slice : am'.slice' el dim) (buf_reader : buf_reader el) : resultm.result' unativeint io_error =
    (!\($'"true; let mut !slice = !slice"') : bool) |> ignore
    !\($'"std::io::Read::read(&mut !buf_reader, &mut !slice)"')

### io_read_by_ref

In [ ]:
inl io_read_by_ref forall t. (lines : lines t) : lines t =
    !\\(lines, $'"std::io::Read::by_ref($0)"')